# Implementing a new CV
In this notebook we will move top-bottom through the structure of the CV classes in `mlcolvar`.

As an example we will write (and comment) step by step the implementation of the `AutoEncoderCV`. 

### Define the class object
In `mlcolvar` CVs class objects inherits from two mother classes:
- `BaseCV` class, which contains some common and default helper functions
- `lightning.LightniningModule` class, which autoamically gives access to the Lightining package utilities 

In the class declaration preamble we set the names of  the `BLOCKS` that consitute the main body of the CV itself. These are initialized as a constant for the class and are thus immutable.  

In [3]:
import torch
import lightning

from mlcolvar.cvs import BaseCV

class AutoEncoderCV(BaseCV, lightning.LightningModule):
    BLOCKS = ['norm_in','encoder','decoder'] 

To keep the code in the library as clear as possible, we should also add short docstring to our CV class briefly explaining how it works!

Anyways to save some space we will skip this in the follwing cells

In [2]:
class AutoEncoderCV(BaseCV, lightning.LightningModule):
    """AutoEncoding Collective Variable. It is composed by a first neural network (encoder) which projects 
    the input data into a latent space (the CVs). Then a second network (decoder) takes 
    the CVs and tries to reconstruct the input data based on them. It is an unsupervised learning approach, 
    typically used when no labels are available.
    Furthermore, it can also be used lo learn a representation which can be used not to reconstruct the data but 
    to predict, e.g. future configurations. 

    For training it requires a DictDataset with the key 'data' and optionally 'weights'. If a 'target' 
    key is present this will be used as reference for the output of the decoder, otherway this will be compared
    with the input 'data'.
    """
    
    BLOCKS = ['norm_in','encoder','decoder'] 

### Define the `__init__` method
The `__init__` method is the signature of the CV model as it initializes all that is necessary for the CV model to run, including blocks, variables, attributes..

Among the elements in the `__init__`, some are common for all of the CVs, while others are characteristic of the CV at hand.

##### General `__init__` elements
- **in/out features**: All the CVs classes in `mlcolvar` should have defined somwhere the number of `in_features` and `out_features`, which are the number of inputs and outputs respectively. They can be passed as parameters to the `__init__` or extracted somewhere else, as we shall see in the next section .
- **options**: The options `dict` provide the interface to modify the defaults of the CV's elements, i.e. parameters of blocks, loss funciton, optimizer..
- ****kwargs**: CVs in `mlcolvar` also accept key-word arguments to be passed to their building functions 

`super().__init__()`


In [ ]:
class AutoEncoderCV(BaseCV, lightning.LightningModule):
    BLOCKS = ['norm_in','encoder','decoder']  

    # ======================== LOOK HERE 0.0 ========================   
    def __init__(self,
                 in_features : int,
                 out_features : int,
                 options : dict = None, 
                 **kwargs):
        super().__init__(in_features=in_features, out_features=out_features, **kwargs)

    # ======================== LOOK HERE 0.0 ========================   